In [1]:
# import Libraries

import openai
import langchain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import os

In [4]:
## Lets Read the document
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [5]:
doc=read_doc('documents/')
len(doc)

58

In [6]:
## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc

In [7]:
documents=chunk_data(docs=doc)
len(documents)

140

In [5]:
## Embedding Technique Of OPENAI
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])


C:\Users\gakum\AppData\Local\Temp\ipykernel_26780\4138089563.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])


In [9]:
vectors=embeddings.embed_query("How are you?")
len(vectors)

1536

In [10]:
documents[1]

Document(metadata={'producer': 'Adobe Acrobat Pro 10.1.16', 'creator': 'Adobe Acrobat Pro 10.1.16', 'creationdate': '2023-02-01T05:28:04+05:30', 'moddate': '2023-02-01T08:28:21+05:30', 'title': '', 'source': 'documents\\budget_speech.pdf', 'total_pages': 58, 'page': 2, 'page_label': '3'}, page_content='CONTENTS \nPART-A \n Page No. \n\uf0b7 Introduction 1 \n\uf0b7 Achievements since 2014: Leaving no one behind 2 \n\uf0b7 Vision for Amrit Kaal – an empowered and inclusive economy 3 \n\uf0b7 Priorities of this Budget 5 \ni. Inclusive Development  \nii. Reaching the Last Mile \niii. Infrastructure and Investment \niv. Unleashing the Potential \nv. Green Growth \nvi. Youth Power  \nvii. Financial Sector \n \n \n \n \n \n \n \n \n \n\uf0b7 Fiscal Management \n24 \nPART B \n  \nIndirect Taxes 27 \n\uf0b7 Green Mobility  \n\uf0b7 Electronics   \n\uf0b7 Electrical   \n\uf0b7 Chemicals and Petrochemicals   \n\uf0b7 Marine products  \n\uf0b7 Lab Grown Diamonds  \n\uf0b7 Precious Metals  \n\uf0b7

In [ ]:
from pinecone import Pinecone
from langchain.vectorstores import Pinecone as pinedb
from langchain.embeddings.openai import OpenAIEmbeddings

# ✅ Set API key and environment variables
PINECONE_API_KEY = ""
PINECONE_INDEX_NAME = ""

# ✅ Correctly initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# ✅ Ensure the index exists before proceeding
available_indexes = [idx.name for idx in pc.list_indexes()]
if PINECONE_INDEX_NAME not in available_indexes:
    raise ValueError(f"Pinecone index '{PINECONE_INDEX_NAME}' does not exist. Check your Pinecone dashboard.")

# ✅ Store documents in Pinecone using LangChain
vectorstore = pinedb.from_documents(
    documents=doc, 
    embedding=embeddings, 
    index_name=PINECONE_INDEX_NAME
)

print("✅ Documents successfully stored in Pinecone!")


d:\Documents\Job_Folder\Langchain_with_deployment\dvenv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


✅ Documents successfully stored in Pinecone!


In [11]:
def retrieve_query(query,k=2):
    matching_results=vectorstore.similarity_search(query,k=k)
    return matching_results

In [12]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [13]:
llm=ChatOpenAI(model="gpt-3.5-turbo",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

C:\Users\gakum\AppData\Local\Temp\ipykernel_22764\849056622.py:2: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain=load_qa_chain(llm,chain_type="stuff")


In [14]:
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [15]:
our_query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answers(our_query)
print(answer)

[Document(metadata={'creationdate': '2023-02-01T05:28:04+05:30', 'creator': 'Adobe Acrobat Pro 10.1.16', 'moddate': '2023-02-01T08:28:21+05:30', 'page': 10.0, 'page_label': '11', 'producer': 'Adobe Acrobat Pro 10.1.16', 'source': 'documents\\budget_speech.pdf', 'title': '', 'total_pages': 58.0}, page_content="7 \n \n \n \nfarmers in contributing to the health of fellow citizens by growing these \n‘Shree Anna’.  \n22. Now to make India a global hub for ' Shree Anna', the Indian Institute \nof Millet Research, Hyderabad will be supported as the Centre of Excellence \nfor sharing best practices, research and technologies at the international \nlevel.    \nAgriculture Credit  \n23. The agriculture credit target will be increased  \nto ` 20 lakh crore with focus on animal husbandry, dairy and fisheries.  \nFisheries \n24. We will launch a new sub-scheme of PM Matsya Sampada Yojana \nwith targeted investment of ` 6,000 crore to further enable activities of \nfishermen, fish vendors, and micr

C:\Users\gakum\AppData\Local\Temp\ipykernel_22764\525972672.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=chain.run(input_documents=doc_search,question=query)


The agriculture credit target will be increased to ₹20 lakh crore.
